In [2]:
import pandas as pd
import numpy as np
import csv

In [12]:
def prep_echo(path, outname):
    #data = pd.read_csv('echosoundertest.txt',sep=" ",header = None)
    df = pd.read_fwf(path)
    # Add 0 in front of numbers that are less than 10
    df['MO'] = df['MO'].apply(lambda x: x+1 if x > 9 else str(x).zfill(2))
    # Create dataid to match sensors, set as index
    df['dataid']=df['YEAR'].astype(str)+'-'+df['MO']+'-'+df['DA'].astype(str)+'T'+df['UTC']
    df.set_index('dataid')
    # Cast to datetime, round to nearest half hour
    df.dataid = pd.to_datetime(df.dataid, format='%Y-%m-%d %H:%M:%S')
    df.dataid = df['dataid'].dt.round('30min') 
    # Drop unnecessary columns
    df = df.drop(['BDMIN', 'BDMAX', 'OBJECT', 'UPINLM', 'PDMIN', 'PDMAX', 'LOG1', 'LOG2', 'CH', 'YEAR', 'MO', 'DA', 'UTC', 'Me/Pl' ], axis=1)
    df.to_csv(f'{outname}.csv', index=False)
    return df

In [15]:
df = prep_echo('leg4_feb.txt', 'leg4_echo_prepped')

0       2022-01-29 07:30:00
1       2022-01-29 07:30:00
2       2022-01-29 07:30:00
3       2022-01-29 07:30:00
4       2022-01-29 07:30:00
                ...        
43211   2022-02-19 21:00:00
43212   2022-02-19 21:00:00
43213   2022-02-19 21:00:00
43214   2022-02-19 21:00:00
43215   2022-02-19 21:00:00
Name: dataid, Length: 43216, dtype: datetime64[ns]

In [ ]:
def iden (row):
    month = str(row[1]) if row[1] > 9 else '0'+str(row[1])
    day = str(row[2]) if row[2] > 9 else '0'+str(row[2])
    
    return str(row[0])+'-'+month+'-'+day+'T'+str(row[3])

In [4]:
nm = df.to_numpy()
count = 0
new = []
for row in nm:
     if row[3]=='08:02:05':
        count += 1
        new.append(row)

sha, inte, deep, total = 0,0,0,0
for i in range(count):
    sha += new[i][8]
    inte+= new[i][9]
    deep+= new[i][7]
    total+=new[i][10]


print(sha, inte, deep, total)

26.923 90.66329999999999 1632.7231000000004 1860.3093000000003


In [46]:
with open("new_file.csv","w+") as my_csv:
    csvWriter = csv.writer(my_csv,delimiter=',')
    csvWriter.writerows(arr)

In [ ]:
print(arr)